In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('max_colwidth', None)

plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline
pd.set_option('display.max_columns', None)

In [104]:
data_string=r"C:\Users\am513\PS-88-21-DEV\lab\aaronfolder\NHIS2009_clean.dta" 
df_og=pd.read_stata(data_string, convert_missing=True)
df=pd.read_stata(data_string)
df_og;
#df.to_csv("NHIS2009_clean.csv")

In [191]:
df_og["hi_hsb1"]=pd.to_numeric(df_og["hi_hsb1"],errors="coerce")
df_og["hi"]=pd.to_numeric(df_og["hi"],errors="coerce")
df_og["age"]=pd.to_numeric(df_og["age"],errors="coerce")#why did this take so long :(

In [192]:
df_og2=df_og.query("marradult == 1 & perweight != 0")
df_og2;

In [193]:
df2=df.query("marradult == 1 & perweight != 0") #keep if marradult==1 & perweight!=0 
df2;

In [194]:
df_og3=df_og2.groupby("serial").sum().reset_index()[["serial","hi_hsb1"]]

In [195]:
df_og4=df_og2.merge(df_og3,how="left",on="serial")
df_og4;

In [196]:
df_og5=df_og4.drop("hi_hsb1_x",axis=1).rename(columns={"hi_hsb1_y":"hi_hsb"})

In [197]:
df_og5_5=df_og5[["serial","hi_hsb"]].set_index("serial")
df_og5_5;#.dtypes #by serial: egen hi_hsb = mean(hi_hsb1)

In [198]:
df_og6=df_og5.dropna(subset=["hi_hsb","hi"],how="any",axis="rows")
df_og6;

In [199]:
males=[
27076,
32501,
36537,
12722,
5958,
18162,
35909,
34396,
17814,
10369,
37415,
36924,
20036,
19066,
22639,
34048,
35904,
29606,
24936,
34048,
21939,
29126,
29542,
2605,
15837,
35956,
22639,
2605,
21666,
2265,
15837,
]

In [200]:
df_og6["female"]=1

In [201]:
df_og6.loc[df_og6["serial"].isin(males),"female"]=0 #by serial: egen female = total(fml)

In [202]:
df_og6["female"].value_counts()

1    29941
0       31
Name: female, dtype: int64

In [203]:
df_og7=df_og6.query("female == 1").drop("female",axis="columns") #keep if female==1     drop female
df_og7;

In [221]:
df_og7.dtypes;

In [216]:
df_og8=df_og7.query("age>=26 & age<=59 & marradult==1 & adltempl>=1")

In [222]:
df_og9=df_og8.groupby(["serial"]).filter(lambda x: len(x)>1) #by serial: gen n = _N   keep if n>1 (This is also really bad code)

In [234]:
husband_somehi_ss=len(df_og8.query("hi == 1 & fml == 0"));
husband_nohi_ss=len(df_og8.query("hi == 0 & fml == 0"));

In [274]:
np.mean(df_og8.query("hi == 1 & fml == 0")["perweight"])

3754.8075752039945

In [277]:
print("Husbands")
pd.DataFrame(data={' ': ["Health index", "Nonwhite","Age","Education","Family Size","Employed","Family Income","Sample Size"],
                   'Some HI (1)': [5, 4,3,4,5,6,7,husband_somehi_ss],
                  'No HI (0)': [3, 4,3,4,5,6,7,husband_nohi_ss],
                  'Difference (3)': [3, 4,3,4,5,6,7," "]})

Husbands


,,Some HI (1),No HI (0),Difference (3)
0,Health index,5,3,3
1,Nonwhite,4,4,4
2,Age,3,3,3
3,Education,4,4,4
4,Family Size,5,5,5
5,Employed,6,6,6
6,Family Income,7,7,7
7,Sample Size,8211,1672,
